In [1]:
import torch
import torch.nn as nn

In [ ]:
''' 
3 DIFFERENT METHODS TO REMEMBER:
 - torch.save(arg, PATH) # can be model, tensor, or dictionary for saving
 - torch.load(PATH)
 - torch.load_state_dict(arg)
'''

In [ ]:
''' 
2 DIFFERENT WAYS OF SAVING
# 1) lazy way: save whole model
torch.save(model, PATH)

# model class must be defined somewhere
model = torch.load(PATH)
model.eval()

- Disadvantage of this approach is that the serialized data is bound to the specific classes and the exact directory structure that is used when the model is saved

# 2) recommended way: save only the state_dict
torch.save(model.state_dict(), PATH)

# model must be created again with parameters
model = Model(*args, **kwargs)
model.load_state_dict(torch.load(PATH)) # It takes loaded dictionary here
model.eval()

- If we just want to save our trained model and use it later for inference then it is enough to only save the parameters, through 2nd approach
'''

In [2]:
class Model(nn.Module):
    def __init__(self, n_input_features):
        super(Model, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)

    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred

In [3]:
model = Model(n_input_features=6)
model

Model(
  (linear): Linear(in_features=6, out_features=1, bias=True)
)

In [ ]:
# train your model...

#################### SAVE ALL ######################################

In [4]:
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[-0.1465,  0.2150,  0.3644,  0.1632,  0.3458,  0.1345]],
       requires_grad=True)
Parameter containing:
tensor([0.2400], requires_grad=True)


In [5]:
# save and load entire model

FILE = "model.pth"
torch.save(model, FILE)

In [6]:
loaded_model = torch.load(FILE)
loaded_model.eval()

Model(
  (linear): Linear(in_features=6, out_features=1, bias=True)
)

In [7]:
for param in loaded_model.parameters():
    print(param)

Parameter containing:
tensor([[-0.1465,  0.2150,  0.3644,  0.1632,  0.3458,  0.1345]],
       requires_grad=True)
Parameter containing:
tensor([0.2400], requires_grad=True)


############ SAVE ONLY STATE DICT #########################

In [10]:
# save only state dict
FILE = "model.pth"
torch.save(model.state_dict(), FILE)

In [11]:
print(model.state_dict())

OrderedDict([('linear.weight', tensor([[-0.1465,  0.2150,  0.3644,  0.1632,  0.3458,  0.1345]])), ('linear.bias', tensor([0.2400]))])


In [12]:
loaded_model = Model(n_input_features=6)
loaded_model.load_state_dict(torch.load(FILE)) # it takes the loaded dictionary, not the path file itself
loaded_model.eval()

Model(
  (linear): Linear(in_features=6, out_features=1, bias=True)
)

In [13]:
print(loaded_model.state_dict())

OrderedDict([('linear.weight', tensor([[-0.1465,  0.2150,  0.3644,  0.1632,  0.3458,  0.1345]])), ('linear.bias', tensor([0.2400]))])


In [14]:
print(model.state_dict())

OrderedDict([('linear.weight', tensor([[-0.1465,  0.2150,  0.3644,  0.1632,  0.3458,  0.1345]])), ('linear.bias', tensor([0.2400]))])


########### LOAD CHECKPOINT #####################

In [15]:
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
print(optimizer.state_dict())

{'state': {}, 'param_groups': [{'lr': 0.01, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'maximize': False, 'foreach': None, 'differentiable': False, 'params': [0, 1]}]}


In [22]:
checkpoint = {
"epoch": 90,
"model_state": model.state_dict(),
"optim_state": optimizer.state_dict()
}

In [18]:
FILE = "checkpoint.pth"
torch.save(checkpoint, FILE)

In [26]:
# Load Checkpoint
model = Model(n_input_features=6)
optimizer = torch.optim.SGD(model.parameters(), lr=0)

In [27]:
model

Model(
  (linear): Linear(in_features=6, out_features=1, bias=True)
)

In [28]:
optimizer

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 0
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)

In [29]:
loaded_checkpoint = torch.load(FILE)
model.load_state_dict(loaded_checkpoint['model_state'])
optimizer.load_state_dict(loaded_checkpoint['optim_state'])
epoch = checkpoint['epoch']
epoch

90

In [30]:
loaded_checkpoint

{'epoch': 90,
 'model_state': OrderedDict([('linear.weight',
               tensor([[-0.1465,  0.2150,  0.3644,  0.1632,  0.3458,  0.1345]])),
              ('linear.bias', tensor([0.2400]))]),
 'optim_state': {'state': {},
  'param_groups': [{'lr': 0.01,
    'momentum': 0,
    'dampening': 0,
    'weight_decay': 0,
    'nesterov': False,
    'maximize': False,
    'foreach': None,
    'differentiable': False,
    'params': [0, 1]}]}}

In [31]:
model.eval()
# - or -
# model.train()

Model(
  (linear): Linear(in_features=6, out_features=1, bias=True)
)

In [32]:
print(optimizer.state_dict())

{'state': {}, 'param_groups': [{'lr': 0.01, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'maximize': False, 'foreach': None, 'differentiable': False, 'params': [0, 1]}]}


In [ ]:
# Remember that you must call model.eval() to set dropout and batch normalization layers 
# to evaluation mode before running inference. Failing to do this will yield 
# inconsistent inference results. If you wish to resuming training, 
# call model.train() to ensure these layers are in training mode.

In [ ]:
""" 
SAVING ON GPU/CPU 

# 1) Save on GPU, Load on CPU
device = torch.device("cuda")
model.to(device)
torch.save(model.state_dict(), PATH)

device = torch.device('cpu')
model = Model(*args, **kwargs)
model.load_state_dict(torch.load(PATH, map_location=device))

# 2) Save on GPU, Load on GPU
device = torch.device("cuda")
model.to(device)
torch.save(model.state_dict(), PATH)

model = Model(*args, **kwargs)
model.load_state_dict(torch.load(PATH))
model.to(device)

# Note: Be sure to use the .to(torch.device('cuda')) function 
# on all model inputs, too!

# 3) Save on CPU, Load on GPU
torch.save(model.state_dict(), PATH)

device = torch.device("cuda")
model = Model(*args, **kwargs)
model.load_state_dict(torch.load(PATH, map_location="cuda:0"))  # Choose whatever GPU device number you want
model.to(device)

# This loads the model to a given GPU device. 
# Next, be sure to call model.to(torch.device('cuda')) to convert the model’s parameter tensors to CUDA tensors
"""